In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data

/Users/elias/Desktop/PTSFC/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
df = get_energy_data()
df = prepare_data(df)

  0%|          | 0/255 [00:00<?, ?it/s]/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 255/255 [00:28<00:00,  8.98it/s]


In [4]:
def remove_time(df, num_years = 0, num_months = 0, num_weeks = 0, num_days = 0, num_hours = 0):
    # date_to_keep = df.index.max() - num_years - num_months - num_days
    date_to_keep = df.index.max() - pd.DateOffset(years = num_years, months=num_months, weeks = num_weeks, days=num_days, hours = num_hours)
    df = df.loc[df.index <= date_to_keep]
    return df

In [5]:
df

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0
...,...,...
2023-11-12 03:00:00,38.68725,6
2023-11-12 04:00:00,39.28950,6
2023-11-12 05:00:00,39.31125,6


In [8]:
from Energy.baseline import baseline

baseline(df, LAST_IDX=-1)

A


,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-13 20:00:00,energy,36 hour,47.099250,54.790000,58.275750,61.938625,68.925319
1,2023-11-14 00:00:00,energy,40 hour,39.612500,43.428562,45.941125,49.032313,56.291488
2,2023-11-14 04:00:00,energy,44 hour,39.472119,41.745313,44.774500,48.597875,55.372306
3,2023-11-14 20:00:00,energy,60 hour,51.757831,55.769188,58.404375,62.398563,69.663931
4,2023-11-15 00:00:00,energy,64 hour,41.628875,44.106687,45.769250,49.245688,56.137538
5,2023-11-15 04:00:00,energy,68 hour,40.345519,42.847750,44.830875,48.450500,55.380406
